In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pickle

# Load necessary datasets
description = pd.read_csv("C:\\Users\\DELLL\\Desktop\\JUPYTER\\medicines\\description.csv")
precautions = pd.read_csv("C:\\Users\\DELLL\\Desktop\\JUPYTER\\medicines\\precautions_df.csv")
medications = pd.read_csv("C:\\Users\\DELLL\\Desktop\\JUPYTER\\medicines\\medications.csv")
diets = pd.read_csv("C:\\Users\\DELLL\\Desktop\\JUPYTER\\medicines\\diets.csv")
workout = pd.read_csv("C:\\Users\\DELLL\\Desktop\\JUPYTER\\medicines\\workout_df.csv")

# Load the training dataset
dataset = pd.read_csv("C:\\Users\\DELLL\\Desktop\\JUPYTER\\Training.csv")
X = dataset.drop('prognosis', axis=1)
y = dataset['prognosis']


In [5]:
# Create symptoms dictionary
symptoms_dict = {symptom: index for index, symptom in enumerate(X.columns)}

# Create diseases list
diseases_list = sorted(y.unique())


In [6]:
# Encode prognosis labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=20)


In [7]:
# Train the SVC model
svc = SVC(kernel='linear')
svc.fit(X_train, y_train)

# Test the model
y_pred = svc.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

# Print model performance
print(f"SVC Accuracy: {accuracy}")

# Save the trained model
pickle.dump(svc, open('svc.pkl', 'wb'))


SVC Accuracy: 1.0


In [8]:
# Load the model for future use
svc = pickle.load(open('svc.pkl', 'rb'))


In [9]:
# Function to predict the disease based on symptoms
def get_predicted_value(patient_symptoms):
    input_vector = np.zeros(len(symptoms_dict))
    for item in patient_symptoms:
        if item in symptoms_dict:
            input_vector[symptoms_dict[item]] = 1
        else:
            print(f"Warning: Symptom '{item}' not found in the dictionary.")
    
    # Reshape input_vector as it needs to be 2D for SVC
    input_vector = np.array(input_vector).reshape(1, -1)
    
    prediction_index = svc.predict(input_vector)[0]
    return diseases_list[prediction_index]


In [10]:
# Helper function to retrieve details about the predicted disease
def helper(dis):
    # Disease description
    desc = description[description['Disease'] == dis]['Description'].values
    desc = " ".join(desc) if len(desc) > 0 else "No description available."

    # Precautions
    pre = precautions[precautions['Disease'] == dis][['Precaution_1', 'Precaution_2', 'Precaution_3', 'Precaution_4']].values
    pre = pre[0] if len(pre) > 0 else ["No precautions available."]

    # Medications
    med = medications[medications['Disease'] == dis]['Medication'].values
    med = med if len(med) > 0 else ["No medications available."]

    # Diet
    die = diets[diets['Disease'] == dis]['Diet'].values
    die = die if len(die) > 0 else ["No diet recommendations available."]

    # Workout
    wrkout = workout[workout['disease'] == dis]['workout'].values
    wrkout = wrkout if len(wrkout) > 0 else ["No workout recommendations available."]

    return desc, pre, med, die, wrkout


In [11]:
# Test the system by accepting user symptoms and predicting the disease
symptoms = input("Enter your symptoms (comma-separated): ")
user_symptoms = [s.strip() for s in symptoms.split(',')]

# Predict the disease
predicted_disease = get_predicted_value(user_symptoms)

# Get the disease details
desc, pre, med, die, wrkout = helper(predicted_disease)

# Output results
print("================= Predicted Disease ==============")
print(predicted_disease)

print("================= Description ==================")
print(desc)

print("================= Precautions ==================")
for i, precaution in enumerate(pre, 1):
    print(f"{i}. {precaution}")

print("================= Medications ==================")
for i, medication in enumerate(med, 1):
    print(f"{i}. {medication}")

print("================= Workout ==================")
for i, workout_recommendation in enumerate(wrkout, 1):
    print(f"{i}. {workout_recommendation}")

print("================= Diets ==================")
for i, diet_recommendation in enumerate(die, 1):
    print(f"{i}. {diet_recommendation}")


Enter your symptoms (comma-separated):  itching,continuous_sneezing,shivering


================= Predicted Disease ==============
Allergy
================= Description ==================
Allergy is an immune system reaction to a substance in the environment.
================= Precautions ==================
1. apply calamine
2. cover area with bandage
3. nan
4. use ice to compress itching
================= Medications ==================
1. ['Antihistamines', 'Decongestants', 'Epinephrine', 'Corticosteroids', 'Immunotherapy']
================= Workout ==================
1. Avoid allergenic foods
2. Consume anti-inflammatory foods
3. Include omega-3 fatty acids
4. Stay hydrated
5. Eat foods rich in vitamin C
6. Include quercetin-rich foods
7. Consume local honey
8. Limit processed foods
9. Include ginger in diet
10. Avoid artificial additives
================= Diets ==================
1. ['Elimination Diet', 'Omega-3-rich foods', 'Vitamin C-rich foods', 'Quercetin-rich foods', 'Probiotics']


C:\Users\DELLL\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
